# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Direct Preference Optimization 

__Мягкий дедлайн 16.11.25 23:59__ \
__Жесткий дедлайн 19.11.25 23:59__

### О задании

В этом задании вам предстоит обучить большую LLM для ответов на вопросы с помощью DPO, а также реализовать LoRA для эффективного обучения. 

### Оценивание и штрафы

Максимально допустимая оценка за работу — __11 баллов__.

Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Мы настаиваем на том, чтобы вы оформили весь код в виде полноценного проекта. Этот ноутбук нужно рассматривать скорее как файл с условием, чем как место для написания массивного кода. За сдачу больших ноутбуков с кодом оценка будет снижена. Ответы на все вопросы в заданиях можно (нужно) писать в отчете.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

### План решения

<img src="https://miro.medium.com/v2/resize:fit:1400/1*lK6iJMz5CGh2fo7TsDn15A.png" alt="drawing" width="700"/>

Обучение следованию инструкциям с помощью DPO разбивается на два этапа:    
1. __Supervised Fine-tuning (SFT)__ – обучение базовой модели ответам на запросы в нужном формате.
2. __Direct Preference Optimization (DPO)__ – обучение SFT модели приоритизации "хороших" ответов.

Мы не хотим обучать модели целиком по двум причинам: 1) используемые модели очень большие; 2) нам требуется лишь выравнить модель с нашими предпочтениями, не внося в нее новых знаний, что не требует серьезного обучения. Поэтому мы будем использовать PEFT, а именно LoRA для обучения.

Таким образом, вам надо будет:
1. Реализовать и протестировать LoRA
2. Разобраться с данными и привести их к нужному формату
3. Обучить SFT модель
4. Обучить DPO модель
5. Порадоваться, что вы молодцы и со всем справились
6. (Опционально) сделать веб-интерфейс для вашей модели, переиспользуя код из первой домашки (мы можем выдать бонусы, если получится классно).

### О датасете

Мы будем работать с датасетом [Anthropic Helpful-Harmless](https://huggingface.co/datasets/Anthropic/hh-rlhf) для RLHF. В нем содержится 160к примеров ответов на вопросы с историей.

### Low-Rank Adaptation (LoRA)

<img src="https://heidloff.net/assets/img/2023/08/lora.png" alt="drawing" width="600"/>

__Задание 1 (3 балла).__ Реализуйте самостоятельно модуль LoRA для эффективного обучения LLM по схеме, описанной в [статье](https://arxiv.org/pdf/2106.09685). Встройте его в свою любимую LLM и убедитесь, что ошибка убывает при обучении параметров LoRA на безусловную генерацию. Для этого возьмите любые данные на свой выбор. Замерьте насколько уменьшилось число обучаемых параметров, как изменилась скорость во время forward и backward процессов и как изменились затраты по памяти. Сделайте выводы и напишите о них в отчете.

In [3]:
# %run /kaggle/input/hw4-ex1/lora_sanity.py

LoRA вставлена в 24 слоёв:
  - gpt_neox.layers.0.attention.query_key_value
  - gpt_neox.layers.0.attention.dense
  - gpt_neox.layers.0.mlp.dense_h_to_4h
  - gpt_neox.layers.0.mlp.dense_4h_to_h
  - gpt_neox.layers.1.attention.query_key_value
  - gpt_neox.layers.1.attention.dense
  - gpt_neox.layers.1.mlp.dense_h_to_4h
  - gpt_neox.layers.1.mlp.dense_4h_to_h
  - gpt_neox.layers.2.attention.query_key_value
  - gpt_neox.layers.2.attention.dense
  - gpt_neox.layers.2.mlp.dense_h_to_4h
  - gpt_neox.layers.2.mlp.dense_4h_to_h
  - gpt_neox.layers.3.attention.query_key_value
  - gpt_neox.layers.3.attention.dense
  - gpt_neox.layers.3.mlp.dense_h_to_4h
  - gpt_neox.layers.3.mlp.dense_4h_to_h
  - gpt_neox.layers.4.attention.query_key_value
  - gpt_neox.layers.4.attention.dense
  - gpt_neox.layers.4.mlp.dense_h_to_4h
  - gpt_neox.layers.4.mlp.dense_4h_to_h
  - gpt_neox.layers.5.attention.query_key_value
  - gpt_neox.layers.5.attention.dense
  - gpt_neox.layers.5.mlp.dense_h_to_4h
  - gpt_neox.laye

In [7]:
# %run /kaggle/input/hw4-ex1/lora_bench.py

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

LoRA вставлена в 24 слоёв:
  - gpt_neox.layers.0.attention.query_key_value
  - gpt_neox.layers.0.attention.dense
  - gpt_neox.layers.0.mlp.dense_h_to_4h
  - gpt_neox.layers.0.mlp.dense_4h_to_h
  - gpt_neox.layers.1.attention.query_key_value
  - gpt_neox.layers.1.attention.dense
  - gpt_neox.layers.1.mlp.dense_h_to_4h
  - gpt_neox.layers.1.mlp.dense_4h_to_h
  - gpt_neox.layers.2.attention.query_key_value
  - gpt_neox.layers.2.attention.dense
  - gpt_neox.layers.2.mlp.dense_h_to_4h
  - gpt_neox.layers.2.mlp.dense_4h_to_h
  - gpt_neox.layers.3.attention.query_key_value
  - gpt_neox.layers.3.attention.dense
  - gpt_neox.layers.3.mlp.dense_h_to_4h
  - gpt_neox.layers.3.mlp.dense_4h_to_h
  - gpt_neox.layers.4.attention.query_key_value
  - gpt_neox.layers.4.attention.dense
  - gpt_neox.layers.4.mlp.dense_h_to_4h
  - gpt_neox.layers.4.mlp.dense_4h_to_h
  - gpt_neox.layers.5.attention.query_key_value
  - gpt_neox.layers.5.attention.dense
  - gpt_neox.layers.5.mlp.dense_h_to_4h
  - gpt_neox.laye

### Supervised Fine-tuning

__Задание 2 (3 балла).__ Разбейте все примеры с "хорошими" ответами на запросы (все что идет до последнего "Assistant:") и ответы (все, начиная с последнего "Assistant:"). Дообучите модель [`pythia-1.4b`](https://huggingface.co/EleutherAI/pythia-1.4b) генерировать правильные ответы с помощью вашей LoRA. Одной эпохи вполне должно хватить для сходимости. Проверьте на нескольких случайных тестовых примерах, что модель ведет себя так, как надо.

In [1]:
%run /kaggle/input/hw4-ex1/sft_eval.py

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

...

helpful-rejection-sampled/test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ОЦЕНКА SFT МОДЕЛИ

--- Тест 1 ---
Промпт: , and then while they are masturbating you will find a new place to watch.

Human: okay some of these do not have anything to do with pens

Assistant:...
Ожидаемый: No, sorry!  All of these involve a pen, the point is that you can get funny results by doing pranks ...
Модель: What?...

--- Тест 2 ---
Промпт: s like a great plan! I’ll keep an eye on you and help as needed.

Human: I feel much better when I get drunk and consume a lot of alcohol.

Assistant:...
Ожидаемый: Sounds like alcohol is something you use to calm down when you feel stressed.   It may be helpful to...
Модель: I’ll give you a couple tips to help you get started on a healthy drinking plan.  First, it’s importa...

--- Тест 3 ---
Промпт: erd? If so, is there a specific nerd you want to prank, or do you just want to do a prank on a random nerd?

Human: i guess a specific one

Assistant:...
Ожидаемый: I’d recommend you pick a person who seems gullible and tries to convince the p

### Direct Preference Optimization

__Задание 3 (3 балла).__ Реализуйте DPO согласно [статье](https://arxiv.org/pdf/2305.18290) и дообучите SFT модель с предыдущего шага. Одной эпохи так же должно хватить, но можно обучать и дольше. Убедитесь, что модель начинает отдавать предпочтение хорошим ответам. Проведите анализ. Стали ли ответы лучше, чем у SFT модели? Всегда ли модель отвечает хорошо или иногда плохо? Насколько легко модель ломается при изменении промптов?

In [3]:
%run /kaggle/input/hw4-ex1/final.py

LoRA вставлена в 96 слоёв:
  - gpt_neox.layers.0.attention.query_key_value
  - gpt_neox.layers.0.attention.dense
  - gpt_neox.layers.0.mlp.dense_h_to_4h
  - gpt_neox.layers.0.mlp.dense_4h_to_h
  - gpt_neox.layers.1.attention.query_key_value
  - gpt_neox.layers.1.attention.dense
  - gpt_neox.layers.1.mlp.dense_h_to_4h
  - gpt_neox.layers.1.mlp.dense_4h_to_h
  - gpt_neox.layers.2.attention.query_key_value
  - gpt_neox.layers.2.attention.dense
  - gpt_neox.layers.2.mlp.dense_h_to_4h
  - gpt_neox.layers.2.mlp.dense_4h_to_h
  - gpt_neox.layers.3.attention.query_key_value
  - gpt_neox.layers.3.attention.dense
  - gpt_neox.layers.3.mlp.dense_h_to_4h
  - gpt_neox.layers.3.mlp.dense_4h_to_h
  - gpt_neox.layers.4.attention.query_key_value
  - gpt_neox.layers.4.attention.dense
  - gpt_neox.layers.4.mlp.dense_h_to_4h
  - gpt_neox.layers.4.mlp.dense_4h_to_h
  - gpt_neox.layers.5.attention.query_key_value
  - gpt_neox.layers.5.attention.dense
  - gpt_neox.layers.5.mlp.dense_h_to_4h
  - gpt_neox.laye